# Building a Bayesian Content Recommender

This is a "free-standing" lab; could go anywhere in the course. I assume you're familiar enough with `stan` to do a linear regression.

## The plan

* Discuss a text triage problem and give some background on contextual muti-armed bandits
* Build a bandit-based recommender. I'll provide the Python code (using `scikit-learn` to generate features and `panel` for a simple GUI); you build the `stan` model to power it
* Do some visualization to get some intuition for how the model does


## The problem

> I just have *too darn many* text documents to read through and I want to prioritize them.

## Why isn't this a standard stats/ML problem?

I mean, we *could* randomly-sample a bunch of documents and label them "interesting" or "not interesting", then fit a model of some sort. But (especially if most documents aren't interesting) this means spending a lot of time reading crap you don't actually care about. And if our problems change day-to-day it means repeating that expensive step every time the problem changes.

Of course, if we make really poor life decisions we could start with a few examples of the types of documents we want, randomly-sample unread documents and label them as negative, and iteratively build a classifier- but then we run the risk of "painting ourself into a corner" where we overfit on details of those examples in a way that doesn't represent our actual interests.

What we really want is a way to balance between **exploring** the big dataset to make sure there aren't types of interesting document we haven't seen yet, and **exploiting** the patterns in documents we've seen so far to spend more of our time on stuff we're more likely to care about.


## So why bring this up in a Bayesian statistics class?

The problem we're describing is an *epistemic uncertainty* about what's interesting in a corpus- we can represent that uncertainty explicitly as a Bayesian regression model that inputs document properties and outputs a guess at whether or not it's interesting (i.e. a logistic regression).

## Bandits

The **multi-armed bandit problem** is a framework for making decisions under uncertainty- in each round, we choose from a set of **actions** (in this case, deciding which document to read), attempting to maximize some **reward** (the number of useful documents read). In the **contextual multi-armed bandit problem**, at each round we also get some information about each decision (for us, this will be the contents of each unread document).

Strategies for solving this problem combine a **model** for predicting the reward for each possible action at each step with a **policy** for choosing an action. One simple example of a policy is called "$\epsilon$-greedy":
* Each round, flip a weighted coin
  * with probability $1-\epsilon$, take whichever action you estimate has the highest likely reward
  * with probability $\epsilon$, choose an action randomly
  
By occasionally choosing a random action, it's easier to hedge against the model overfitting to cases we've seen before (at the cost of most of those random actions probably not being high-reward).

For today's lab, we're going to use a Bayesian policy called **Thompson sampling:**
* Build some model $f$ with parameters $\beta$ that inputs a context vector $x$, and outputs an estimate of the reward: $\hat{r} = f(x; \beta)$
* At each round:
  * Compute a posterior distribution over the model parameters using the context vectors and rewards you've seen so far: $P(\beta |D) \propto P(D|\beta)P(\beta)$, where $D$ is the dataset of results of previous rounds.
  * Sample a value from the posterior $\beta^{'} \sim P(\beta |D)$
  * Estimate the rewards of all possible actions using that sample of your model parameters: $\hat{r}_{i}^{'} = f(x_{i}; \beta^{'})$
  * Choose the action with the highest estimated reward
  
When we've just started and don't have many observations- our posterior should be very broad and the sampling will be more-or-less random. As we get more data and start constraining the posterior, the samples will gradually focus more and more tightly around the high-reward cases.

To implement this, we'll use a latent semantic index for $x$ and a Bayesian logistic regression for $f$.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import pystan
import json
import sklearn.feature_extraction, sklearn.decomposition, sklearn.manifold
from tqdm import tqdm
import panel as pn

%matplotlib inline
from IPython.core.pylabtools import figsize

In [ ]:
pn.extension()

## Load the data

I used Kaggle's dataset of DOJ press releases: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases

But use whatever repository of text you want. Go nuts.

In [ ]:
datafile = open("/Users/joe/Documents/tldr/doj_press_releases.json", "r").read()
lines = datafile.strip().split("\n")
parsed = [json.loads(l) for l in tqdm(lines)]
df = pd.DataFrame(parsed)
print(len(df))

In [ ]:
df.head()

## Preparing context vectors

For this lab, we'll use some old-school NLP to build a "bag of words" representation of our data:

* Build a **term-document matrix**, where each row represents a document and each column represents a token (e.g. a word). Each cell of the matrix is the number of times that word appears in that document.
* Rescale using **term-frequency-inverse-document-frequency** (TF-IDF); that is, reduce the value of each matrix cell by the log of the fraction of documents containing that word (to de-emphasize words like "a" and "the" that appear everywhere so they're not useful for differentiating).
* Use a **truncated singular-value decomposition** to compress the matrix from (num_documents, num_tokens) to (num_documents, d) where $d <<$ num_tokens. This way we can deal with a few dozen dimensions instead of tens of thousands- this technique applied to text is called **latent semantic indexing** or **latent semantic analysis**.
* Finally, we'll normalize all the rows of our LSI matrix to an L2 norm of 1.

In [ ]:
%%time
tdm = sklearn.feature_extraction.text.TfidfVectorizer().fit_transform(df.contents.values)
tdm.shape

In [ ]:
%%time
d = 50 #25
components = sklearn.decomposition.TruncatedSVD(n_components=d).fit_transform(tdm)
components.shape

In [ ]:
components = components/(np.sqrt((components**2).sum(axis=1).reshape(-1,1))+1e-5)
components.shape

Initialize an empty array of labels:

In [ ]:
labels = np.nan*np.ones(components.shape[0])

## Build the `stan` model

Time for you to start pulling your weight around here. Throw together a simple logistic regression.

In [ ]:
model_code = """
data {
  int N;  // number of observations
  int<lower=0,upper=1> y[N];
  int K; // number of covariates
  matrix [N, K] X; // design matrix (your data)
}
parameters {
  // write your own damn parameters
}
model {
  //write your own damn model
}
"""

In [ ]:
# TEST VERSION- DELETE THIS FOR THE LAB
model_code = """
data {
  int N;  // number of observations
  int<lower=0,upper=1> y[N];
  int K; // number of covariates
  matrix [N, K] X; // design matrix
}
parameters {
  real alpha;
  vector[K] beta;
}
model {
  // priors
  alpha ~ normal(0., 2.);
  beta ~ normal(0., 2.);
  
  y ~ bernoulli_logit_glm(X, alpha, beta);
}
"""

In [ ]:
%%time
model = pystan.StanModel(model_code=model_code)

## Build the actual recommender

To speed things along I've coded up the boring stuff- I've added a ton of comments so you can see that I'm not doing anything terribly complicated in here.

In [ ]:
def choose_next_index(num_samples=1000):
    """
    Function to choose the index of the next document to read,
    using Thompson sampling (if you have at least one positive and
    one negative example) or random sampling otherwise.
    
    :num_samples: how many MCMC samples to run
    """
    # find the indices of labeled and unlabeled documents
    unlabeled = np.isnan(labels)
    labeled_indices = np.arange(len(labels))[~unlabeled]
    unlabeled_indices = np.arange(len(labels))[unlabeled]
    # if we don't have a positive and a negative example,
    # randomly pick a document
    if min((labels==0).sum(), (labels==1).sum()) == 0:
        return np.random.choice(unlabeled_indices)
    # select and index using Thompson sampling
    else:
        # run the Stan model
        datadict = {"N":len(labeled_indices),
            "K":components.shape[1], 
            "X":components[labeled_indices,:], 
            "y":labels[labeled_indices].astype(int)}
        fit = model.sampling(data=datadict, iter=num_samples, chains=1)
        # extract the last sample of the model parameters
        betas = fit.extract()["beta"]
        beta_sample = betas[-1,:].reshape(-1,1)
        # use the model parameters to estimate the reward for each document
        # (since we only care about the ordering we don't need to calculate
        # the sigmoid)
        reward_estimates = (components.dot(beta_sample)).ravel()
        # set the reward estimates to -1000 for the documents we've
        # already read so that none of them get flagged as the max value
        reward_estimates[labeled_indices] = -1000
        # return the index of the highest-estimated-reward document
        return reward_estimates.argmax()

## Recommender GUI

Let's use `panel` to throw together a quick-and-dirty GUI. It'll have a few components:

* a markdown panel to display the document
* buttons to feed back that the document was or wasn't useful
* a tiny markdown panel to keep track of the total number of useful and non-useful documents so far
* under the hood: a `sample()` function that calls `choose_next_index()` and updates the markdown panels
* under the hood: callback functions for the two buttons, that records the 1 or 0 and then calls `sample()`

In [ ]:
class Recommenderizer(object):
    """
    Class for a GUI to manage the recommender
    """
    
    def __init__(self):
        self._index_history = []
        self._label_history = []
        self.yesbutton = pn.widgets.Button(name="thanks I love it", button_type="primary")
        self.nobutton = pn.widgets.Button(name="thanks I hate it", button_type="danger")
        self.counter = pn.pane.Markdown("Yes: %s \n\nNo: %s"%((labels ==1).sum(), (labels==0).sum()))
        self.text_pane = pn.pane.Markdown("foo", width=800)
        
        self.yesbutton.on_click(self._record_yes)
        self.nobutton.on_click(self._record_no)
        self._sample()
      
    
    def _sample(self):
        """
        Choose the next index and update the displayed document
        """
        self.next_index = choose_next_index()
        self.text_pane.object = "## %s \n %s"%(df.title.values[self.next_index], 
                                               df.contents.values[self.next_index].replace("\xa0", "\n"))
        self.counter.object = "Yes: %s \n\nNo: %s"%((labels ==1).sum(), (labels==0).sum())
        
    def _record_yes(self, *event):
        """
        Here's everything that happens when the user hits the "yes" button- change the
        document's label to 1, record the label and index to the history, and set up the next sample
        """
        labels[self.next_index] = 1
        self._index_history.append(self.next_index)
        self._label_history.append(1)
        self._sample()
        
    def _record_no(self, *event):
        """
        Here's everything that happens when the user hits the "no" button- change the
        document's label to 0, record the label and index to the history, and set up the next sample
        """
        labels[self.next_index] = 0
        self._index_history.append(self.next_index)
        self._label_history.append(0)
        self._sample()
        
    def __call__(self):
        """
        Return the panel GUI
        """
        return pn.layout.Column(pn.layout.Row(self.yesbutton, self.nobutton, self.counter), 
                                self.text_pane)
    

Now initialize a recommender and call it to pop up the GUI:

In [ ]:
foo = Recommenderizer()

In [ ]:
foo()

## Visualizing results

Let's look at a few things:

* use T-SNE to build a scatter plot of all the documents, marking the documents we've read so we can see how we're exploring the corpus
* sample posterior values of the model parameters and look at the marginal credible intervals
* the cumulative **regret**- the difference between the reward from our system and the reward a perfect system would have generated

### Scatter plot

T-SNE ("t-distributed stochastic neighbor embedding") is an algorithm for trying to embed high-dimensional data into a lower-dimensional space while preserving the relative distances between data points.

It won't be a *perfect* representation of our data but is a reasonably quick way to get a picture of the overall structure of our corpus. TSNE will take a couple minutes to compute so just run this cell once:

In [ ]:
%%time
embeddings = sklearn.manifold.TSNE().fit_transform(components)

This cell should be much faster; you can re-run it as your bandit learns. We'll plot:

* unlabeled documents with a dot
* documents you labelled "not useful" with an X
* documents you labelled "useful" with a triangle

In [ ]:
figsize(12,8)
unlabeled = np.isnan(labels)

plt.plot(embeddings[unlabeled,0], embeddings[unlabeled, 1], ".", alpha=0.05)
plt.plot(embeddings[labels==0,0], embeddings[labels==0,1], "x")
plt.plot(embeddings[labels==1,0], embeddings[labels==1,1], "^")
plt.axis("off");

## Model parameters

Let's draw some samples from the model posterior, and plot the means and credible intervals for each parameter.

How many dimensions in the latent semantic space is it really using? How much do the distributions overlap?

In [ ]:
unlabeled = np.isnan(labels)
labeled_indices = np.arange(len(labels))[~unlabeled]

datadict = {"N":len(labeled_indices),
            "K":components.shape[1], 
            "X":components[labeled_indices,:], 
            "y":labels[labeled_indices].astype(int)}
beta_samples = model.sampling(data=datadict, iter=5000, chains=4).extract()["beta"]

In [ ]:
means = beta_samples.mean(axis=0)
cred_int = np.stack([np.quantile(beta_samples[:,i], [0.05,0.95]) for i in range(beta_samples.shape[1])]).T

In [ ]:
figsize(7,5)
plt.errorbar(np.arange(beta_samples.shape[1]), means, yerr=np.abs(cred_int-means.reshape(1,-1)), fmt="o")
plt.xlabel("$\\beta$", fontsize=14)
plt.ylabel("posterior mean and \n90% credible interval", fontsize=14)
plt.grid("on");

## Regret plot

Regret analysis is a standard tool for evaluating the performance of a bandit algorithm. If we assume we're working in the case where there are still interesting documents left to be read (i.e., if the model doesn't recommend a useful document it isn't because we've already got them all), then an ideal model would always give a reward of 1.

If we're at step $k$, then, and have found $l$ useful documents so far, then the regret is $k-l$.

In [ ]:
plt.plot(np.cumsum([1-x for x in foo._label_history]), lw=2)
plt.plot(np.arange(len(foo._label_history))+1, "--", lw=2)
plt.grid(True)
plt.xlabel("round", fontsize=14)
plt.ylabel("cumulative regret", fontsize=14);

## So what's next?

Build a better model. This would be an interesting place to try out a horseshoe or spike-and-slab prior. Or use priors with heavier tails. Or just change the variance on the coefficient priors.

## Closing thoughts

OK, so can we go ahead and build a production system out of this model? There's a few considerations I'd look at before I start making promises:

### Efficiency

Thompson sampling with `stan` is wonderfully flexible but may be impractical when you scale up. There are more computationally-expedient policies that accomplish more-or-less the same thing. Simple heuristics like $\epsilon$-greedy are difficult to beat in practice, and `LinUCB` uses the same Bayesian model as we used here with a policy that doesn't require MCMC (*A Contextual-Bandit Approach to Personalized News Article Recommendation* by Li *et al*, 2010).

If anyone's interested I think I've got a Python implementation of `LinUCB` floating around somewhere.

### Data drift

We've implicitly assumed here that our data is being drawn from a stationary distribution- in real life that's seldom the case (which is good for us if we don't want our jobs to be automated). Does our use case need us to reuse today's recommender tomorrow? If so, how do we extend the model (or integrate with a heuristic) to make sure a human sees new types of documents (which might otherwise accidentally get labelled as uninteresting)?


### Feature space

We used a latent semantic index as the input to our recommender - the SVD is a convenient way to try and summarize the global statistics of the corpus, but might struggle if we're looking for something specific. For example, it won't do a great job with rare tokens (names, phone numbers, email addresses, etc) or concepts that require multiple tokens to express (names, titles, addresses, etc). 

Some thought should be put into what information needs to be filtered and how to engineer a reasonable feature space. Techniques like automated entity extraction, distributed semantic embeddings (GloVe, word2vec), and locality-sensitive hashing could be used here.

### Redundancy

If the corpus contains a lot of documents with redundant information, we could run into trouble: the model could return stuff we already know because (by definition) it's similar to the documents we said were useful. We've modeled the reward for each document as independent, but in real life reading the same thing more than once isn't terribly useful.

Other people have put thought into this and some solutions exit- check out *Linear Submodular Bandits and their Application to Diversified Retrieval* by Yue and Guestrin, NIPS (2011).

### Boolean search

Remember, if you know enough about what you're looking for that you can articulate it with a SQL query, that's *way* easier than turning it into a machine learning problem.

## What if I wanted to prototype a system like this for a real problem?

My advice would be to find a proxy dataset that you have labels for (either finding a pre-made dataset or assembling one from different corpuses). That way you can simulate how the bandit learns to set your hyperparameters (how many dimensions for the truncated SVD? what prior variance to use on coefficients? etc) and test potential concern areas (what if the user makes mistakes? what if we add new data from a slightly different distribution?)